In [301]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
from scipy import signal
from scipy.fft import fft, fftfreq
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler

## Initial pre-processing:

In [331]:
# sampling rate
FS = 100 

# import and clean data
path_kin = 'data/subject1_smile_1_landmarks.csv'
raw = pd.read_csv(path_kin)

raw['time'] = raw['Frame_number'] / FS
raw = raw.iloc[::-1]
raw = raw.set_index('time')

#create new df for storing processed data
data = pd.DataFrame(index = raw.index)

# Find displacement of target markers
pos_top = raw[['landmark_62_x','landmark_62_y']].to_numpy()
pos_bottom = raw[['landmark_66_x','landmark_66_y']].to_numpy()
data['deltaV'] = np.linalg.norm(pos_top - pos_bottom, axis=1)

pos_left = raw[['landmark_51_x','landmark_51_y']].to_numpy()
pos_right = raw[['landmark_57_x','landmark_57_y']].to_numpy()
data['deltaH'] = np.linalg.norm(pos_right - pos_left, axis=1)

# scale data from 0 to 1
scaler = MinMaxScaler()
data.deltaH = scaler.fit_transform(data.deltaH.values.reshape(-1,1))
data.deltaV = scaler.fit_transform(data.deltaV.values.reshape(-1,1))

## 1. Compare lowpass filter at different cuttoff frequencies

In [315]:
cols_low = ['Raw','20 Hz*','22.5 Hz','25_Hz','30 Hz','32.5 Hz','35 Hz']
Vlow = [(data.deltaV.values)] # adding raw data
Hlow = [data.deltaH.values] # adding raw data

passes = [20,22.5,25,30,32.5,35]

ORDER = 2
FS = 100 
lcoeff = []
#cols = []
for p in passes:
    lcoeff.append(sp.signal.bessel(ORDER, Wn = p/(FS / 2), btype = 'lowpass'))
    #cols.append(f'{p} Hz')


for c in range(len(lcoeff)): # will apply first coeff first to both deltaV/H, then append 
    #print(c)
    Vlow.append(sp.signal.filtfilt(lcoeff[c][0],lcoeff[c][1],data['deltaV']))
    Hlow.append(sp.signal.filtfilt(lcoeff[c][0],lcoeff[c][1],data['deltaH']))

Vlow = pd.DataFrame(np.asarray(Vlow).transpose(),columns = cols_low) 
Hlow = pd.DataFrame(np.asarray(Hlow).transpose(),columns = cols_low) 

##### skipme
%matplotlib notebook
plot(Vlow)

Observation:
- All of the filters give similar trends for temporal resolution. The 25 Hz signal seems to retain more shape in when activating versus others. 
- None of the lowpass filters performed very well in removing noise when the subject is holding the 'smile' position. 
- The 25 Hz filter collects more relevant information regarding motor neuron activation

## 2. Find ideal kernel size for median filter

In [305]:
cols_med = ['Raw','17 N','19 N','21_N*','23_N','25 N']
Vmed = [(data.deltaV.values)] # adding raw data
Hmed = [(data.deltaH.values)] # adding raw data

ksize = [17,19,21,23,25]

FS = 100 
lcoeff = []
#cols = []
for k in range(len(ksize)):
    Vmed.append(sp.signal.medfilt(data['deltaV'],ksize[k]))
    Hmed.append(sp.signal.medfilt(data['deltaH'],ksize[k]))

Vmed = pd.DataFrame(np.asarray(Vmed).transpose(),columns = cols_med) 
Hmed = pd.DataFrame(np.asarray(Hmed).transpose(),columns = cols_med) 

##### skipme
%matplotlib notebook
plot(Hmed)

Observation: 
- little difference in activation times between kernel sizes. Median filter is more appropriate to eliminate outliers in the data. Suggest it has more ability at performing with better temporal resolution versus spatial resolution, as some outliers can still be seen, especially with the lower end (17) and higher-end (25) kernel sizes. 

## 3. Compare filter (1), (2), and original filter selected in paper. 
selecting lowpass 25 Hz and median k = 23

In [328]:
V = pd.DataFrame(index=data.index)
H = pd.DataFrame(index=data.index)

V['Raw'] = Vmed['Raw'].values 
V['Median'] = Vmed['23_N'].values
V['Lowpass'] = Vlow['25_Hz'].values

H['Raw'] = Hmed['Raw'].values 
H['Median'] = Hmed['23_N'].values
H['Lowpass'] = Hlow['25_Hz'].values

##### skipme
%matplotlib notebook
plot(V)

## Extra: Plotting function

In [332]:
def plot(df):
    cols = list(df.columns)
    for col in df.columns:
        plt.plot(df.index,df[col],'--o')
        plt.grid()
        plt.title('Filter Comparison')
        plt.xlabel('Sample')
        plt.ylabel('Normalized Distance')
        plt.legend(cols,loc='best')
        plt.grid()